In [1]:
#Import Dependencies

import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import numpy as np
import pymongo
import psycopg2 as pg
import psycopg2
import config as login

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.nhl_db
collection = db.articles

In [4]:
csv_file = "Resources/Population.csv"
pop_df = pd.read_csv(csv_file)
pop_df.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,...,NRANK_PPCHG2010,NRANK_PPCHG2011,NRANK_PPCHG2012,NRANK_PPCHG2013,NRANK_PPCHG2014,NRANK_PPCHG2015,NRANK_PPCHG2016,NRANK_PPCHG2017,NRANK_PPCHG2018,NRANK_PPCHG2019
0,10,0,0,0,United States,308758105,309321666,311556874,313830990,315993715,...,X,X,X,X,X,X,X,X,X,X
1,20,1,0,0,Northeast Region,55318443,55380134,55604223,55775216,55901806,...,3,3,3,4,4,4,4,4,4,4
2,20,2,0,0,Midwest Region,66929725,66974416,67157800,67336743,67560379,...,4,4,4,3,3,3,3,3,3,3
3,20,3,0,0,South Region,114563030,114866680,116006522,117241208,118364400,...,1,1,1,1,1,1,1,1,1,1
4,20,4,0,0,West Region,71946907,72100436,72788329,73477823,74167130,...,2,2,2,2,2,2,2,2,2,2


In [5]:
#Delete unnecessary columns
new_pop=pop_df[['REGION','NAME','POPESTIMATE2018']].copy()
new_pop.head()

,REGION,NAME,POPESTIMATE2018
0,0,United States,326687501
1,1,Northeast Region,56046620
2,2,Midwest Region,68236628
3,3,South Region,124569433
4,4,West Region,77834820


In [6]:
new_pop = new_pop.set_index('REGION')

In [7]:
rds_connection_string = "postgres:R31n33nj!@localhost:5432/Flu_Crew"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [8]:
engine.table_names()

['Population_Data', 'Influenza_Data']

In [11]:
new_pop.to_sql(name='Population_Data', con=engine, if_exists='append', index=False)

In [12]:
pd.read_sql_query('SELECT * FROM "Population_Data"', con=engine).head()

,Region,NAME,POPESTIMATE2018
0,None,United States,326687501
1,None,Northeast Region,56046620
2,None,Midwest Region,68236628
3,None,South Region,124569433
4,None,West Region,77834820


In [13]:
flu_file = "Resources/influenza_data.csv"
flu_df = pd.read_csv(flu_file)
flu_df.head()

,REGION TYPE,REGION,YEAR,WEEK,% WEIGHTED ILI,%UNWEIGHTED ILI,AGE 0-4,AGE 25-49,AGE 25-64,AGE 5-24,AGE 50-64,AGE 65,ILITOTAL,NUM. OF PROVIDERS,TOTAL PATIENTS
0,States,Alabama,2017,40,X,1.75721,X,X,X,X,X,X,698,85,39722
1,States,Alaska,2017,40,X,2.72675,X,X,X,X,X,X,141,17,5171
2,States,Arizona,2017,40,X,2.3554,X,X,X,X,X,X,406,42,17237
3,States,Arkansas,2017,40,X,0.812294,X,X,X,X,X,X,37,17,4555
4,States,California,2017,40,X,1.75421,X,X,X,X,X,X,852,102,48569


In [14]:
new_flu=flu_df[["REGION","YEAR","AGE 0-4", "AGE 25-49", "AGE 5-24", "AGE 50-64", "AGE 65", "ILITOTAL", "TOTAL PATIENTS"]].copy()
new_flu.head()          

,REGION,YEAR,AGE 0-4,AGE 25-49,AGE 5-24,AGE 50-64,AGE 65,ILITOTAL,TOTAL PATIENTS
0,Alabama,2017,X,X,X,X,X,698,39722
1,Alaska,2017,X,X,X,X,X,141,5171
2,Arizona,2017,X,X,X,X,X,406,17237
3,Arkansas,2017,X,X,X,X,X,37,4555
4,California,2017,X,X,X,X,X,852,48569


In [18]:
new_pop.to_sql(name='Influenza_Data', con=engine, if_exists='replace', index=True)

In [19]:
pd.read_sql_query('SELECT * FROM "Influenza_Data"', con=engine).head()

,REGION,NAME,POPESTIMATE2018
0,0,United States,326687501
1,1,Northeast Region,56046620
2,2,Midwest Region,68236628
3,3,South Region,124569433
4,4,West Region,77834820
